In [1]:
import soundfile
import torch
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text
from espnet2.asr.frontend.default import *
from espnet.nets.pytorch_backend.transformer.subsampling import *

In [2]:
d = ModelDownloader()
ASR_MODEL_PATH = "/home/jmpark/espnet/tools/anaconda/envs/espnet_1.7/lib/python3.8/site-packages/espnet_model_zoo/653d10049fdc264f694f57b49849343e/exp/asr_train_asr_transformer_e18_raw_bpe_sp/54epoch.pth"
LM_MODEL_PATH = "/home/jmpark/espnet/tools/anaconda/envs/espnet_1.7/lib/python3.8/site-packages/espnet_model_zoo/653d10049fdc264f694f57b49849343e/exp/lm_train_lm_adam_bpe/17epoch.pth"

In [3]:
speech2text = Speech2Text(
    **d.download_and_unpack('Shinji Watanabe/librispeech_asr_train_asr_transformer_e18_raw_bpe_sp_valid.acc.best'),
    # Decoding parameters are not included in the model file
    maxlenratio=0.0,
    minlenratio=0.0,
    beam_size=1,
    ctc_weight=0.4,
    lm_weight=0.6,
    penalty=0.0,
    nbest=1
)

net = speech2text.asr_model

In [4]:
print(net)

ESPnetASRModel(
  (frontend): DefaultFrontend(
    (stft): Stft(n_fft=512, win_length=512, hop_length=128, center=True, normalized=False, onesided=True)
    (frontend): Frontend()
    (logmel): LogMel(sr=16000, n_fft=512, n_mels=80, fmin=0, fmax=8000.0, htk=False)
  )
  (specaug): SpecAug(
    (time_warp): TimeWarp(window=5, mode=bicubic)
    (freq_mask): MaskAlongAxis(mask_width_range=[0, 30], num_mask=2, axis=freq)
    (time_mask): MaskAlongAxis(mask_width_range=[0, 40], num_mask=2, axis=time)
  )
  (normalize): GlobalMVN(stats_file=/home/jmpark/espnet/tools/anaconda/envs/espnet_1.7/lib/python3.8/site-packages/espnet_model_zoo/653d10049fdc264f694f57b49849343e/exp/asr_stats_raw_sp/train/feats_stats.npz, norm_means=True, norm_vars=True)
  (encoder): TransformerEncoder(
    (embed): Conv2dSubsampling6(
      (conv): Sequential(
        (0): Conv2d(1, 512, kernel_size=(3, 3), stride=(2, 2))
        (1): ReLU()
        (2): Conv2d(512, 512, kernel_size=(5, 5), stride=(3, 3))
        (3): 

In [5]:
for n, m in net.named_modules():
    print(n)


frontend
frontend.stft
frontend.frontend
frontend.logmel
specaug
specaug.time_warp
specaug.freq_mask
specaug.time_mask
normalize
encoder
encoder.embed
encoder.embed.conv
encoder.embed.conv.0
encoder.embed.conv.1
encoder.embed.conv.2
encoder.embed.conv.3
encoder.embed.out
encoder.embed.out.0
encoder.embed.out.1
encoder.embed.out.1.dropout
encoder.encoders
encoder.encoders.0
encoder.encoders.0.self_attn
encoder.encoders.0.self_attn.linear_q
encoder.encoders.0.self_attn.linear_k
encoder.encoders.0.self_attn.linear_v
encoder.encoders.0.self_attn.linear_out
encoder.encoders.0.self_attn.dropout
encoder.encoders.0.feed_forward
encoder.encoders.0.feed_forward.w_1
encoder.encoders.0.feed_forward.w_2
encoder.encoders.0.feed_forward.dropout
encoder.encoders.0.feed_forward.activation
encoder.encoders.0.norm1
encoder.encoders.0.norm2
encoder.encoders.0.dropout
encoder.encoders.1
encoder.encoders.1.self_attn
encoder.encoders.1.self_attn.linear_q
encoder.encoders.1.self_attn.linear_k
encoder.encoder

In [6]:
encoder_input = []
def hook(module, input, output):
    print("This is input")
    encoder_input.append(input)

for n, m in net.named_modules():
    if n == 'encoder.encoders.0':
        m.register_forward_hook(hook)

In [7]:
TEST_DATA_PATH = "/home/jmpark/espnet/egs2/librispeech/asr1/data/dev_clean"
WAV_LIST_PATH = TEST_DATA_PATH + "/wav.scp"

file_name_list = []

with open(WAV_LIST_PATH, "r") as f:
    lines = f.readlines()
    for line in lines:
        num, name = line.split(' ')
        file_name_list.append(name[:-1])

In [8]:
audio_num = 5
speech, rate = soundfile.read(file_name_list[audio_num])

In [9]:
out = speech2text(speech)

This is input


In [13]:
print(speech.shape)
print(encoder_input[0][0].shape)

(144160,)
torch.Size([1, 187, 512])


In [ ]:
# input shape [1, seqlen, d_model]
